## Load slices of binary data 

In [1]:
import numpy as np
from glob import glob
import pyqtgraph as pq
from fish.image.vol import get_stack_dims, filter_flat, unfilter_flat
%gui qt

In [2]:
# turn 4D array into 3D
def unroll(v):
    if v.ndim == 3:
        return v.reshape(v.shape[0] * v.shape[1], v.shape[2])
    elif v.ndim == 4:        
        return v.reshape(v.shape[0], v.shape[1] * v.shape[2], v.shape[3])

# seed correlation
def seed_corr(seed, data):
    from numpy import where, correlate, prod, zeros
    from scipy.stats import zscore
 
    flat = data.reshape(data.shape[0], prod(data.shape[1:])).T
    valid = where(flat.std(1) != 0)[0]
    seed = zscore(seed).copy()
    normed = zscore(flat[valid, :], axis=1)
    
    corrs = zeros(flat.shape[0])
    
    for ind, val in enumerate(valid):
        corrs[val] = correlate(seed, normed[ind]) / len(seed)

    corrs = corrs.reshape(data.shape[1:])
    
    return corrs

# Need this function to prevent pyqtgraph from leaking memory
def wipe_ims(pq):
    from numpy import zeros
    for iw in pq.images:
      iw.setImage(zeros((1,1))) #<- dummy 1x1 image
      iw.parent().close()

In [21]:
base_dir = 'F:/davis/20170728/20170728_7dpf_gfapiglusnfr_gfapreachr_f1_594_test_4_20170728_114037/'
dims = tuple(get_stack_dims(base_dir)[::-1])
fnames = glob(base_dir + '*.stack')
fnames.sort()
print(np.r_[len(fnames), dims])

##################
if dims[0]!=50 and dims[0]!=1:
    single_plane = False
else:
    single_plane = True
    
print("single plane: " + str(single_plane))
##################

mem = [np.memmap(f, dtype='uint16', shape=dims) for f in fnames]

[ 144   50 1024 2048]
single plane: True


In [22]:
# load some ephys
#ep_path = 'W:/YuMu/SPIM/active_datasets/20170727/fish1/fish1/6dpf_gfapiglusnfr_gfap_reachr_opto_3/6dpf_gfapiglusnfr_gfap_reachr_opto_3.10chFlt'
ep_path = None
if ep_path:
    from fish.ephys.ephys import load
    ep_dat = load(ep_path)
    
    from fish.ephys.ephys import estimate_onset, windowed_variance
    swims = windowed_variance(ep_dat[0])[0]
    cam = estimate_onset(ep_dat[2], 2.8, 10)

In [23]:
%%time
wipe_ims(pq)
plr = slice(0, 400)
ds_xy =2 
if single_plane:
    plr_true = np.arange(plr.start, plr.stop * dims[0])
dat = np.array([m[:,::ds_xy,::ds_xy] for m in mem[plr]])

Wall time: 2min 15s


In [24]:
if single_plane:
    if dat.ndim > 3:
        dat = dat.reshape(dat.shape[0] * dat.shape[1], dat.shape[2], dat.shape[3])

if dat.ndim > 3:
    im = pq.image(unroll(dat))
else:
    im = pq.image(dat)

In [22]:
rois = {}

In [23]:
rois['waist'] = im.roi.getArrayRegion(dat, im.imageItem, axes=(1,2)).mean((1,2))

In [36]:
from scipy.stats import zscore
pl = pq.plot(cam[plr_true], zscore(rois['waist']), pen='r')
x_axis_ep  = range(cam[plr_true[0]], cam[plr_true[-1]])
pl.plot(x_axis_ep, zscore(swims[x_axis_ep]))

In [40]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.plot(ep_dat[0])
plt.axvline(cam[0])
plt.axvline(cam[-1])
plt.show()

In [ ]:

pl = pq.plot(cam, 5 + zscore(rois['motor']), color='m')
pl.plot(zscore(epdat[0]), color='y')

In [ ]:
# load ephys
from fish.ephys.ephys import load, windowed_variance
ep_fname = 'W:/davis/data/ephys/20170502/6dpf_cy302xcy171_f2_omr_2.10chFlt'
epdat = load(ep_fname)

In [ ]:
from fish.ephys.ephys import estimate_onset

In [ ]:
cam = estimate_onset(epdat[2], 3.7, 100)[:len(fnames)]

In [ ]:
smth = [windowed_variance(v)[0] for v in epdat[0:2]]

In [39]:
import matplotlib.pyplot as plt
%matplotlib
plt.plot(smth[1])
plt.show()

Using matplotlib backend: Qt4Agg


NameError: name 'smth' is not defined

In [ ]:
pq.plot(epdat[1][cam[0]:cam[-1]][::10])